## Pipeline (HGSOC)

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from datetime import date

/home/haz19024/.venv/work/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
df = pd.read_csv('../ov2295_snv_counts.csv')
df = df.astype({'chrom': 'str', 
                'coord': 'int32', 
                'ref': 'str',
                'alt': 'str',
                'ref_counts': 'int32',
                'alt_counts': 'int32'})
df['snp_id'] = df['chrom'] + ':' + df['coord'].astype(str) + ':' + df['ref'] + ':' + df['alt']

/home/haz19024/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# filter out sites with low coverage
# def filtering(df, min_total_counts=0, ):
#     df = df[df['total_counts']>min_total_counts]
#     return df

# false positive: sequencing error, usually bewteen 0.01 to 0.1
# false negative: combined error from amplication, sequencing and dropout, we assume 0.2 or even higher
# genotype prior: flat prior, gt can be either 0 or 1
def posterior_prob_with_fn_fp_flat_ht(ref_counts, alt_counts, fp=0.01, fn=0.2, prior_ref=0.5, margin=1e-5):
    # binary case : haplotype
    p00, p01, p10, p11 = 1-fp, fp, fn, 1-fn
    g0 = prior_ref * (p00**ref_counts) * (p01**alt_counts)
    g1 = (1-prior_ref) * (p10**ref_counts) * (p11**alt_counts)
    prob =  max(min(np.round(g0/(g0+g1), 5), 1-margin), margin)
    return prob

def posterior_prob_with_fn_fp_flat_gt(ref_counts, alt_counts, fp=0.01, fn=0.2, prior_ref=0.5, margin=1e-5):
    # trinary case : genotype
    p00, p01, p10, p11 = 1-fp, fp, fn, 1-fn
    l00 = p00**ref_counts * p01**alt_counts
    l11 = p10**ref_counts * p11**alt_counts
    l01 = (0.5*(p00+p10))**ref_counts * (0.5*(p01+p11))**alt_counts
    g00 = prior_ref**2 * l00
    g01 = 2*(1-prior_ref)*prior_ref * l01
    g11 = (1-prior_ref)**2 * l11
    prob =  max(min(np.round(g00/(g00+g01+g11), 5), 1-margin), margin)
    return prob


def phred_likelihood_with_fn_fp_flat(ref_counts, alt_counts, fp=0.01, fn=0.2):
    # Q-phred score = -10 * log10(p)
    p00, p01, p10, p11 = 1-fp, fp, fn, 1-fn
    l00 = p00**ref_counts * p01**alt_counts
    l11 = p10**ref_counts * p11**alt_counts
    l01 = (0.5*(p00+p10))**ref_counts * (0.5*(p01+p11))**alt_counts
    q00 = int(-10 * np.log10(l00))
    q11 = int(-10 * np.log10(l11))
    q01 = int(-10 * np.log10(l01))
    return q00, q01, q11


In [4]:
# check the number of cell ids for each sample
for sample in df['sample_id'].unique():
    print(sample, df[df['sample_id'] == sample]['cell_id'].unique().shape[0])

SA921 616
SA922 588
SA1090 739


In [5]:
# filter out samples with preclustered cells
df_clones = pd.read_csv('../ov2295_clone_clusters.csv')
filtered_cells = df_clones['cell_id'].unique()
# filtered df
df = df[df['cell_id'].isin(filtered_cells)]

In [6]:
# prelimary view on data
cells = df['cell_id'].drop_duplicates().tolist()
sites = df['snp_id'].drop_duplicates().tolist()
cells = sorted(cells)
sites = sorted(sites)
print(len(cells), len(sites))

891 14068


In [62]:
# # filter out non-informative sites, criterion: more than 650 cells missed
# informative_df = df.groupby('snp_id').size().sort_values(ascending=False)
# informative_sites = informative_df[informative_df > len(cells) - 650].index.tolist()
# df = df[df['snp_id'].isin(informative_sites)]

In [7]:
# check again the number of cells and the number of sites, in HUNTRESS, there are 891 cells with 744 mutations, a slight difference
cells = df['cell_id'].drop_duplicates().tolist()
sites = df['snp_id'].drop_duplicates().tolist()
print(len(cells), len(sites))
prefix = f'ov2295_{len(sites)}_{len(cells)}'
print(prefix)

891 14068
ov2295_14068_891


In [8]:
# create a meta dataframe for the data
df_site_meta = df[['snp_id', 'chrom', 'coord', 'ref', 'alt']].drop_duplicates()
df_site_meta.sort_values(by='snp_id', inplace=True)
df_site_meta['id'] = [f's{ix}' for ix in np.arange(df_site_meta.shape[0])]
# df_site_meta.to_csv(f'./{prefix}_meta_site.csv', index=False)

In [9]:
df_cell_meta = df[['cell_id', 'sample_id']].drop_duplicates().sort_values(by='cell_id')
df_cell_meta['id'] = [ix for ix in np.arange(df_cell_meta.shape[0])]
df_cell_meta = pd.merge(df_cell_meta, df_clones, on='cell_id')
# df_cell_meta.to_csv(f'./{prefix}_meta_cell.csv', index=False)

In [10]:
# get af for each site
def get_ml_gt(ref_counts, alt_counts):
    a, b, c = phred_likelihood_with_fn_fp_flat(ref_counts, alt_counts)
    min_phred = min(a, b, c) 
    if a == min_phred:
        return 0
    elif b == min_phred:
        return 1
    else:
        return 2

df['ml_gt'] = df.apply(lambda x: get_ml_gt(x['ref_counts'], x['alt_counts']), axis=1)

In [11]:
# calcutate the phred likelihood for each site
def get_phred_likelihood(ref_counts, alt_counts, ml_gt):
    gts = ['0/0', '0/1', '1/1']
    a, b, c = phred_likelihood_with_fn_fp_flat(ref_counts, alt_counts)
    return f'{gts[ml_gt]}:{a},{b},{c}'
df['phred'] = df.apply(lambda x: get_phred_likelihood(x['ref_counts'], x['alt_counts'], x['ml_gt']), axis=1)

In [12]:
def af(gts):
    counts = [0, 0, 0]
    for gt in gts:
        counts[gt] += 1
    counts = np.array(counts)
    freqs = counts / sum(counts)
    return np.round(freqs[0] + 0.5 * freqs[1], 2)

allele_freqs = df.groupby('snp_id')['ml_gt'].agg(func=af).to_frame().rename(columns={'ml_gt': 'af'})
# del df['af']
# del df_site_meta['af']
df = df.join(allele_freqs, on='snp_id')
df_site_meta = df_site_meta.join(allele_freqs, on='snp_id')
# df_site_meta.to_csv('./meta_site.csv', index=False)

In [13]:
# calculate the posterior probability for each site
df['genotype_prob'] = df.apply(lambda x: posterior_prob_with_fn_fp_flat_gt(x['ref_counts'], x['alt_counts'], prior_ref=x['af']), axis=1)

In [14]:
def write_to_scistree(genotype_matrix):
    nsite, ncell = genotype_matrix.shape
    output = f'ov2295_{nsite}_{ncell}.scistree'
    with open(output, 'w') as out:
        out.write(f'HAPLOID {nsite} {ncell}')
        for i in range(ncell):
            out.write(f' {i}')
        out.write('\n')
        # for i in range(self.nsite):
        # idx = 1
        for i in range(nsite):
            out.write(f's{i}')
            # idx += 1
            for j in range(ncell):
                prob = genotype_matrix[i, j]
                out.write(f' {prob:.5f}')
            out.write('\n')

def write_to_huntress(matrix):
    flags = [0, 1, 1, 3]
    nsite, ncell = matrix.shape
    output = f'ov2295_{nsite}_{ncell}.huntress'
    with open(output, 'w') as out:
        out.write(f'cellID/mutID')
        for i in range(nsite):
            out.write(f'\ts{i}')
        out.write('\n')
        for i in range(ncell):
            out.write(f'c{i}')
            for j in range(nsite):
                val = flags[int(matrix[j, i])]
                out.write(f'\t{val}')
            out.write('\n')

def write_to_cellphy(matrix):
    nsite, ncell = matrix.shape
    output = f'ov2295_{nsite}_{ncell}.vcf'
    header = \
f'''##fileformat=VCFv4.3
##fileDate={date.today().isoformat()}
##source=ov2295
##ncell={ncell}
##nsite={nsite}
##reference=NONE
##contig=<ID=1>
##phasing=NO
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Phread-scaled genotype likelihoods">
'''
    with open(output, 'w') as out:
        out.write(header)
        out.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT')
        for i in range(ncell):
            out.write(f'\t{i}')
        out.write('\n')
        for i in range(nsite):
            chrom, coord, ref, alt, af, snp_id = df_site_meta.iloc[i][['chrom', 'coord', 'ref', 'alt', 'af', 'snp_id']]
            out.write(f"{chrom}\t{coord}\t{snp_id}\t{ref}\t{alt}\t.\tPASS\tAF={af}\tGT:PL")
            for j in range(ncell):
                out.write(f'\t{matrix[i, j]}')
            out.write('\n')                               

In [71]:
# # formatted as scistree input
# scistree_matrix_df = df.pivot(index='snp_id', columns='cell_id', values='genotype_prob')
# scistree_matrix = scistree_matrix_df.to_numpy()
# genotype_matrix_scistree = np.nan_to_num(scistree_matrix, nan=0.5)
# write_to_scistree(genotype_matrix_scistree) 

In [72]:
# # formatted as huntress input
# huntress_matrix_df = df.pivot(index='snp_id', columns='cell_id', values='ml_gt')
# huntress_matrix = huntress_matrix_df.to_numpy()
# huntress_matrix = np.nan_to_num(huntress_matrix, nan=3)
# write_to_huntress(huntress_matrix)

In [15]:
# formatted as cellphy input
cellphy_matrix_df = df.pivot(index='snp_id', columns='cell_id', values='phred') 
cellphy_matrix_df.fillna('./.:0,0,0', inplace=True)
cellphy_matrix = cellphy_matrix_df.to_numpy()
write_to_cellphy(cellphy_matrix)
# cellphy_matrix_df

In [174]:
# # run scistree 
# input_scis = f'{prefix}.scistree'
# ! /usr/bin/time -o {prefix}.scistree.time /data/haotian/scistree/scistree_bin/Code/Scistree-v2.1.1.1/scistree -v -T 30 -e -o {prefix}.prob.muttree.gml $input_scis > {prefix}.scistree.log

In [75]:
# # run huntress
# input_huntress = f'{prefix}.huntress'
# ! /usr/bin/time -o {prefix}.huntress.time python /home/haz19024/softwares/HUNTRESS/HUNTRESS.py --i $input_huntress --o {prefix}.SC --t 30 > {prefix}.HUNTRESS.log

In [76]:
# # run cellphy
# input_cellphy = f'{prefix}.vcf'
# ! /usr/bin/time -o {prefix}.cellphy.time /home/haz19024/softwares/cellphy/cellphy.sh FAST -a -t 30 -r $input_cellphy > {prefix}.cellphy.log 2>&1

## Results (HGSOC)

In [16]:
# annovar annotation
humandb = '/home/haz19024/softwares/annovar/humandb/humandb/'
xref = '/home/haz19024/softwares/annovar/example/gene_fullxref.txt'
!perl /home/haz19024/softwares/annovar/table_annovar.pl {prefix}.vcf $humandb -buildver hg19 -out {prefix} -remove -protocol refGene,cytoBand,exac03,avsnp147,dbnsfp30a -operation gx,r,f,f,f -nastring . -vcfinput -polish -xref $xref


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 ov2295_14068_891.vcf > ov2295_14068_891.avinput>
NOTICE: Finished reading 14079 lines from VCF file
NOTICE: A total of 14068 locus in VCF file passed QC threshold, representing 14068 SNPs (5707 transitions and 8361 transversions) and 0 indels/substitutions
NOTICE: Finished writing allele frequencies based on 12534588 SNP genotypes (5084937 transitions and 7449651 transversions) and 0 indels/substitutions for 891 samples

NOTICE: Running with system command </home/haz19024/softwares/annovar/table_annovar.pl ov2295_14068_891.avinput /home/haz19024/softwares/annovar/humandb/humandb/ -buildver hg19 -outfile ov2295_14068_891 -remove -protocol refGene,cytoBand,exac03,avsnp147,dbnsfp30a -operation gx,r,f,f,f -nastring . -polish -xref /home/haz19024/softwares/annovar/example/gene_fullxref.txt -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing op

In [316]:
# gene annotation
df_gene_annotation = pd.read_table(f'./{prefix}.hg19_multianno.txt')
df_gene_annotation['variant_id'] = df_gene_annotation['Chr'].astype(str) + ':' + df_gene_annotation['Start'].astype(str) + ':' + df_gene_annotation['Ref'] + ':' + df_gene_annotation['Alt']
df_gene_annotation.set_index('variant_id', inplace=True)
df_gene_annotation

/home/haz19024/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (0,72) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo894,Otherinfo895,Otherinfo896,Otherinfo897,Otherinfo898,Otherinfo899,Otherinfo900,Otherinfo901,Otherinfo902,Otherinfo903
variant_id,,,,,,,,,,,,,,,,,,,,,
10:100129846:C:A,10,100129846,100129846,C,A,intergenic,LOXL4;PYROXD2,dist=101895;dist=13479,.,.,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","1/1:20,3,0"
10:100185101:A:G,10,100185101,100185101,A,G,intronic,HPS1,.,.,.,...,"./.:0,0,0","1/1:40,7,1","0/0:0,2,6","./.:0,0,0","./.:0,0,0","1/1:40,7,1","./.:0,0,0","./.:0,0,0","./.:0,0,0","0/0:0,4,13"
10:100209326:T:C,10,100209326,100209326,T,C,ncRNA_intronic,LOC101927278,.,.,.,...,"./.:0,0,0","0/0:0,2,6","./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
10:100283942:C:T,10,100283942,100283942,C,T,intronic,HPSE2,.,.,.,...,"0/0:0,4,13","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","0/0:0,2,6","./.:0,0,0","0/0:0,2,6"
10:10086005:G:T,10,10086005,10086005,G,T,intergenic,LOC101928272;LINC02670,dist=748449;dist=14680,.,.,...,"./.:0,0,0","0/0:0,2,6","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X:98278474:C:A,X,98278474,98278474,C,A,intergenic,DIAPH2;PCDH19,dist=1418479;dist=1268168,.,.,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
X:98452192:T:C,X,98452192,98452192,T,C,intergenic,DIAPH2;PCDH19,dist=1592197;dist=1094450,.,.,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
X:98700894:C:T,X,98700894,98700894,C,T,intergenic,DIAPH2;PCDH19,dist=1840899;dist=845748,.,.,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","1/1:20,3,0","./.:0,0,0"


In [299]:
# find cancer-driver genes 
df_gene_exonic = df_gene_annotation[df_gene_annotation['Func.refGene'] == 'exonic']
print(sorted(df_gene_exonic['Gene.refGene'].unique()))
df_gene_exonic


['ABCA13', 'ABCA9', 'ACBD3', 'ADAM12', 'ADGB', 'ADGRF3', 'ADRA2B', 'AGRN', 'AKAP9', 'ALPK1', 'APOB', 'APOL1', 'APOL3', 'ARAP3', 'ARID1A', 'ARNT', 'ASB15', 'ASIC2', 'ASL', 'ATAD2B', 'ATXN2L', 'BIRC6', 'BTBD11', 'C2orf42', 'CACNA1G', 'CASR', 'CBL', 'CCDC180', 'CCDC77', 'CDC123', 'CDH23', 'CHD2', 'CLDN17', 'CRISPLD1', 'CSMD3', 'CUX2', 'DEPDC5', 'DGKQ', 'DHRS13', 'DNAH17', 'DNAH2', 'DNAH9', 'DNHD1', 'DUOX1', 'ECM2', 'EDAR', 'EFNA3', 'EFS', 'EGR2', 'ELAPOR1', 'EMILIN2', 'EML2', 'ERN2', 'ESPNL', 'ESRRG', 'FAM124A', 'FAM170B', 'FAM83H', 'FBXO32', 'FBXW5', 'FCRL3', 'FNDC1', 'FOXP2', 'GAB4', 'GANC', 'GDF7', 'GEM', 'GHR', 'GLG1', 'GNAT1', 'GRIK1', 'GRIK3', 'GRIK5', 'GRIN3A', 'GSTM3', 'GUCA1ANB', 'HDGFL1', 'HOOK3', 'HTR1D', 'HTRA4', 'INSL4', 'INTS5', 'IRF4', 'JPH1', 'KANK1', 'KANSL1', 'KANSL1L', 'KDM2B', 'KIF1A', 'KIZ', 'KLHL33', 'LAMA2', 'LPCAT1', 'LRRC31', 'LRRC3C', 'LTN1', 'LY6D', 'MAGEB10', 'MAP1S', 'MARCOL', 'MCOLN2', 'MDN1', 'MED12L', 'METTL18', 'MMAA', 'MRPL21', 'MTREX', 'MYH9', 'NAT10', '

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo894,Otherinfo895,Otherinfo896,Otherinfo897,Otherinfo898,Otherinfo899,Otherinfo900,Otherinfo901,Otherinfo902,Otherinfo903
147,10,11912073,11912073,G,A,exonic,PROSER2,.,nonsynonymous SNV,PROSER2:NM_153256:exon4:c.G976A:p.A326T,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
169,10,12257806,12257806,T,C,exonic,CDC123,.,nonsynonymous SNV,CDC123:NM_006023:exon5:c.T305C:p.F102S,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","1/1:20,3,0","./.:0,0,0","0/0:0,4,13","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
185,10,127730736,127730736,C,T,exonic,ADAM12,.,nonsynonymous SNV,"ADAM12:NM_001288974:exon19:c.G2179A:p.E727K,AD...",...,"./.:0,0,0","./.:0,0,0","0/0:0,2,6","0/0:0,4,13","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
417,10,50339712,50339712,T,C,exonic,FAM170B,.,synonymous SNV,FAM170B:NM_001164484:exon2:c.A798G:p.E266E,...,"0/0:0,9,27","./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
512,10,64573400,64573400,G,A,exonic,EGR2,.,nonsynonymous SNV,"EGR2:NM_000399:exon2:c.C998T:p.T333M,EGR2:NM_0...",...,"1/1:20,3,0","./.:0,0,0","./.:0,0,0","0/0:0,4,13","./.:0,0,0","0/0:0,4,13","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13610,9,95267903,95267903,G,C,exonic,ECM2,.,nonsynonymous SNV,"ECM2:NM_001197295:exon7:c.C1310G:p.A437G,ECM2:...",...,"./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
13613,9,95838138,95838138,G,A,exonic,SUSD3,.,nonsynonymous SNV,"SUSD3:NM_001287006:exon2:c.G161A:p.G54E,SUSD3:...",...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
13843,X,24593304,24593304,C,G,exonic,PCYT1B,.,nonsynonymous SNV,"PCYT1B:NM_001163264:exon7:c.G786C:p.E262D,PCYT...",...,"1/1:40,7,1","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","1/1:40,7,1","./.:0,0,0"
13855,X,27839987,27839987,T,C,exonic,MAGEB10,.,synonymous SNV,MAGEB10:NM_182506:exon3:c.T564C:p.C188C,...,"0/0:0,2,6","./.:0,0,0","./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"


In [99]:
# original paper used the following genes
marker_genes = ['TP53', 'FOXP2', 'ZHX1', 'HTR1D', 'INSL4', 'C7orf10', 'SUGCT']
df_marker_genes = df_gene_annotation[df_gene_annotation['Gene.refGene'].isin(marker_genes)]
df_marker_genes = df_marker_genes[df_marker_genes['Func.refGene'] == 'exonic']
df_marker_genes['variant_id'] = df_marker_genes.apply(func=lambda x: str(x['Chr'])+':'+str(x['Start'])+ ':' + x['Ref'] + ':' + x['Alt'], axis=1)
df_marker_genes = df_marker_genes[['variant_id', 'Gene.refGene']]
df_marker_genes

,variant_id,Gene.refGene
3839,17:7578265:A:G,TP53
5179,1:23520184:G:A,HTR1D
11388,7:114329967:G:A,FOXP2
11789,7:40900013:T:C,SUGCT
12200,8:124266354:T:A,ZHX1
13443,9:5231710:C:T,INSL4


In [19]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import dollo
import pickle
import popgen

In [97]:
with open('../ov2295_tree.pickle', 'rb') as f:
    tree = pickle.load(f)
df_nodes = pd.read_csv('../ov2295_nodes.csv')
df_mutations = pd.merge(df_nodes, df_site_meta, left_on='variant_id', right_on='snp_id')
df_mutations = df_mutations.groupby(['variant_id']).apply(lambda x: x[x['ml_origin']==1])
df_mutations

,,variant_id,node,loss,origin,presence,ml_origin,ml_presence,ml_loss,snp_id,chrom,coord,ref,alt,id,af
variant_id,,,,,,,,,,,,,,,,
10:100129846:C:A,9,10:100129846:C:A,9,3.985202e-41,0.993210,0.006790,1,1,0,10:100129846:C:A,10,100129846,C,A,s0,0.88
10:100185101:A:G,26,10:100185101:A:G,9,3.896468e-67,0.993189,0.006811,1,1,0,10:100185101:A:G,10,100185101,A,G,s1,0.83
10:100209326:T:C,34,10:100209326:T:C,0,0.000000e+00,1.000000,0.000000,1,1,0,10:100209326:T:C,10,100209326,T,C,s2,0.38
10:100283942:C:T,60,10:100283942:C:T,9,5.161910e-86,0.993218,0.006782,1,1,0,10:100283942:C:T,10,100283942,C,T,s3,0.86
10:10086005:G:T,70,10:10086005:G:T,2,5.890891e-61,0.993219,0.006781,1,1,0,10:10086005:G:T,10,10086005,G,T,s4,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X:98278474:C:A,239083,X:98278474:C:A,12,1.019073e-35,0.791074,0.005422,1,1,0,X:98278474:C:A,X,98278474,C,A,s14063,0.89
X:98452192:T:C,239093,X:98452192:T:C,5,4.754889e-44,0.993219,0.006781,1,1,0,X:98452192:T:C,X,98452192,T,C,s14064,0.80
X:98700894:C:T,239105,X:98700894:C:T,0,0.000000e+00,1.000000,0.000000,1,1,0,X:98700894:C:T,X,98700894,C,T,s14065,0.00


In [100]:
df_marker_genes.set_index('variant_id', inplace=True)
df_mutations.set_index('variant_id', inplace=True)
df_marker_genes = df_marker_genes.join(df_mutations, on='variant_id')
df_marker_genes.set_index('id', inplace=True)

In [20]:
import re
# read outputs
def get_scistree_genotype(prefix):
    geno_file = f'{prefix}.scistree.genos.imp'
    genotypes = []
    with open(geno_file, 'r') as f:
        for line in f.readlines():
            if line.startswith('Site'):
                line = line.strip()
                genos = line.split('\t')[1].split()
                genos = list(map(int, genos))
                genotypes.append(genos)
    return np.array(genotypes)

def get_huntress_genotype(prefix):
    geno_file = f'{prefix}.SC.CFMatrix'
    genotypes = pd.read_table(geno_file, index_col=0).values.T
    return genotypes

def read_mutation_list(file):
    edges = {}
    with open(file, 'r') as f:
        for line in f.readlines():
            items = line.strip().split()
            if int(items[1]) > 0:
                eid, num_muts, muts = items
                edges[eid] = [int(_[1:]) for _ in muts.split(',')]
            else:
                eid = items[0]
                edges[eid] = []
    return edges


def read_mutation_tree(file):
    with open(file, 'r') as f:
        line = f.readline()
        line = line.strip()
    return popgen.utils.from_newick(line)

def get_cellphy_genotype(prefix):
    raxml_tree = f'{prefix}.vcf.Mapped.raxml.mutationMapTree'
    raxml_list = f'{prefix}.vcf.Mapped.raxml.mutationMapList'
    num_snp, num_cell = len(sites), len(cells)
    genotype = np.zeros([num_snp, num_cell], dtype=int)
    edges = read_mutation_list(raxml_list)
    tree = read_mutation_tree(raxml_tree)
    generator = popgen.utils.TraversalGenerator(order='post')
    for node in generator(tree):
        if not node.is_root():
            eid = re.search(r'\[(\d+)\]', node.branch).group(1)
            muts = edges[eid]
            # muts = [mut-1 for mut in muts] # to 0-indexed
            leaves = [int(leaf.name) for leaf in node.get_leaves()] # to 0-indexed
            genotype[np.ix_(muts, leaves)] = 1
    return genotype

In [21]:
# get scistree genotype
scistree_genotype = get_scistree_genotype(prefix)

In [269]:
def is_covered(clade1, clade2):
    diff = clade1 - clade2
    if 0 in diff and 1 in diff and -1 in diff:
        return 0 # no relationship
    if 1 in diff:
        return 1 # clade1(snp1) happens before clade2(snp2)
    else:
        return -1 # clade2(snp2) happens before clade1(snp1)
    
def get_ancestor_descendant_pairs_4_given_genes(geno, df_marker_genes):
    genes = df_marker_genes.index.tolist()
    num_snp, _ = geno.shape
    mutations = {df_marker_genes.loc[gene, 'Gene.refGene']: set() for gene in genes}
    for i in range(len(genes)):
        for j in range(len(genes)):
            if i == j:
                continue
            gene1, gene2 = genes[i], genes[j]
            gene1_idx = int(gene1.split('s')[1])
            gene2_idx = int(gene2.split('s')[1])
            code = is_covered(geno[gene1_idx], geno[gene2_idx])
            if code == 1:
                mutations[df_marker_genes.loc[gene1, 'Gene.refGene']].add(df_marker_genes.loc[gene2, 'Gene.refGene'])
            elif code == -1:
                mutations[df_marker_genes.loc[gene2, 'Gene.refGene']].add(df_marker_genes.loc[gene1, 'Gene.refGene'])
    return mutations


In [270]:
# interested mutations order, Ancestral mutations with significant impact were found in TP53 (584T>C), FOXP2, and SUGCT
# Clade specific mutations were found in ZHX1, HTR1D, INSL4
scistree_mutations = get_ancestor_descendant_pairs_4_given_genes(scistree_genotype, df_marker_genes)
scistree_mutations


{'TP53': {'FOXP2', 'HTR1D', 'INSL4', 'SUGCT', 'ZHX1'},
 'HTR1D': {'INSL4'},
 'FOXP2': {'HTR1D', 'INSL4', 'SUGCT', 'TP53', 'ZHX1'},
 'SUGCT': {'FOXP2', 'HTR1D', 'INSL4', 'TP53', 'ZHX1'},
 'ZHX1': set(),
 'INSL4': set()}

In [333]:
# find the mutations in the tree
marker_gene_clades = {}
for gene in marker_genes:
    ix = df_marker_genes[df_marker_genes['Gene.refGene'] == gene].index.tolist()
    if ix:
        genotype = scistree_genotype[int(ix[0][1:])]
        # find 1s in the genotype
        clade = np.where(genotype == 1)[0]
        marker_gene_clades[gene] = set(clade)

# marker_gene_clades['INSL4']


In [318]:
df_gene_annotation.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo894,Otherinfo895,Otherinfo896,Otherinfo897,Otherinfo898,Otherinfo899,Otherinfo900,Otherinfo901,Otherinfo902,Otherinfo903
variant_id,,,,,,,,,,,,,,,,,,,,,
10:100129846:C:A,10,100129846,100129846,C,A,intergenic,LOXL4;PYROXD2,dist=101895;dist=13479,.,.,...,"./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","1/1:20,3,0"
10:100185101:A:G,10,100185101,100185101,A,G,intronic,HPS1,.,.,.,...,"./.:0,0,0","1/1:40,7,1","0/0:0,2,6","./.:0,0,0","./.:0,0,0","1/1:40,7,1","./.:0,0,0","./.:0,0,0","./.:0,0,0","0/0:0,4,13"
10:100209326:T:C,10,100209326,100209326,T,C,ncRNA_intronic,LOC101927278,.,.,.,...,"./.:0,0,0","0/0:0,2,6","./.:0,0,0","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"
10:100283942:C:T,10,100283942,100283942,C,T,intronic,HPSE2,.,.,.,...,"0/0:0,4,13","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","0/0:0,2,6","./.:0,0,0","0/0:0,2,6"
10:10086005:G:T,10,10086005,10086005,G,T,intergenic,LOC101928272;LINC02670,dist=748449;dist=14680,.,.,...,"./.:0,0,0","0/0:0,2,6","0/0:0,2,6","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0","./.:0,0,0"


In [328]:
# find the exonic mutations at the same branch
def locate_variant(ix):
    variant_id = df_mutations.iloc[ix].snp_id
    variant = df_gene_annotation.loc[variant_id]
    return variant 

def find_all_muts_at_same_branch(clade):
    all_muts = set()
    for site, geno in enumerate(scistree_genotype):
        c = set(np.where(geno == 1)[0])
        if clade == c:
            variant = locate_variant(site)
            if variant['Func.refGene'] == 'exonic':
                all_muts.add(variant['Gene.refGene'])
            # all_muts.add(variant['Gene.refGene'])

    return all_muts

ancestral_genes = find_all_muts_at_same_branch(marker_gene_clades['TP53'])
print(len(ancestral_genes))

49


In [331]:
# find cluster-specific mutations
def find_cluster_genes(min_leaf=100):
    clade_genes = {}
    for i, genotype in enumerate(scistree_genotype):
        num_leaf = sum(genotype)
        if num_leaf > min_leaf:
            variant = locate_variant(i)
            if variant['Func.refGene'] == 'exonic':
                clade_genes[variant['Gene.refGene']] = num_leaf
    return clade_genes

cluster_genes = find_cluster_genes()
for gene in cluster_genes:
    print(gene, cluster_genes[gene])

PROSER2 244
CDC123 891
ADAM12 464
FAM170B 891
EGR2 220
TYSND1 891
CDH23 234
VCL 219
PATE2 425
NAT10 891
INTS5 891
PACS1 234
BTBD11 213
CUX2 145
SLC6A12 891
PARP11 221
SPRYD3 891
CCDC77 224
PDS5B 891
ZC3H13 222
FAM124A 891
PCDH17 215
ZNF839 237
KLHL33 424
RPGRIP1 891
EFS 413
SYNE2 188
PROX2 218
TCL1A 214
GANC 891
DUOX1 891
TLN2 218
ZNF710 464
CHD2 170
SYNM 243
SMG1 464
ZG16B 203
ATXN2L 891
RNF40 162
GLG1 171
ZNF469 891
DNAH9 216
DHRS13 891
PHF12 220
ASIC2 235
LRRC3C 218
TOP2A 207
KANSL1 224
CACNA1G 184
ABCA9 427
NEURL4 109
TP53 891
DNAH2 222
EMILIN2 213
RPRD1A 416
WDR7 199
MAP1S 423
TLE2 229
RYR1 891
GRIK5 156
ZNF404 200
EML2 413
GSTM3 145
ARNT 413
EFNA3 234
FCRL3 194
RASAL2 891
ESRRG 216
ACBD3 420
HTR1D 243
RYR2 891
OR2L13 189
ARID1A 168
GRIK3 198
AGRN 195
KIZ 243
PTPRA 891
NCOA6 413
PROCR 891
LTN1 235
GRIK1 210
CLDN17 211
GAB4 192
APOL3 241
APOL1 420
MYH9 891
NPTXR 891
SHISAL1 891
EDAR 240
NEB 131
SGO2 891
PARD3B 198
KANSL1L 891
SLC19A3 891
ESPNL 226
ATAD2B 167
KIF1A 891
ADGRF3 244
BI

In [103]:
# time (secs) 47min 41.49s
time = 47 * 60 + 41.49
time


2861.49

## Tree Vis

In [123]:
pptree = popgen.utils.build_perfect_phylogeny(scistree_genotype.T)
df_cell_meta.set_index('id', inplace=True)

In [243]:
# use iTOL webserver to visualize the tree
# https://itol.embl.de/
# generate annotation file
# 1-indexed cell ids
import seaborn as sns
palette = sns.color_palette("tab10").as_hex()
clones = df_cell_meta['clone_id'].unique()
clone_colors = {clone: palette[ix] for ix, clone in enumerate(clones)}
# define my color palette
a = sns.color_palette("coolwarm", 18).as_hex()
b = sns.light_palette("seagreen").as_hex()
clone_colors = {10: a[0],
                11: a[1],
                1: a[5],
                8: a[6],
                0: b[1],
                17:b[4],
                6:a[13],
                16:a[15],
                14:a[17]
}

sample_colors = {'SA1090': palette[0], 
                 'SA921': palette[1],
                 'SA922': palette[2],}
def generate_itol_annotation_clone(pptree, df_cell_meta):
    with open(f'{prefix}.clone.itol.txt', 'w') as f:
        f.write('DATASET_COLORSTRIP\n')
        f.write('SEPARATOR COMMA\n')
        f.write('DATASET_LABEL,OV2295\n')
        f.write('COLOR,#ff0000\n')
        f.write('DATA\n')
        for leaf in pptree.get_leaves():
            clone = df_cell_meta.loc[leaf, 'clone_id']
            f.write(f'{leaf+1},{clone_colors[clone]},clone{clone}\n')

def generate_itol_annotation_sample(pptree, df_cell_meta):
    with open(f'{prefix}.sample.itol.txt', 'w') as f:
        f.write('DATASET_COLORSTRIP\n')
        f.write('SEPARATOR COMMA\n')
        f.write('DATASET_LABEL,OV2295\n')
        f.write('COLOR,#ff0000\n')
        f.write('DATA\n')
        for leaf in pptree.get_leaves():
            sample = df_cell_meta.loc[leaf, 'sample_id']
            f.write(f'{leaf+1},{sample_colors[sample]},{sample}\n')

In [244]:
generate_itol_annotation_clone(pptree, df_cell_meta)
generate_itol_annotation_sample(pptree, df_cell_meta)

In [337]:
# clone colors
clone_colors, sample_colors

({10: '#4a63d3',
  11: '#5a78e4',
  1: '#a3c2fe',
  8: '#b6cefa',
  0: '#c5decf',
  17: '#54a075',
  6: '#f59f80',
  16: '#e57058',
  14: '#c73635'},
 {'SA1090': '#1f77b4', 'SA921': '#ff7f0e', 'SA922': '#2ca02c'})

In [336]:
# find number of cells in each clone 
df_clones.groupby('clone_id').size()

clone_id
0     145
1      99
6      96
8     110
10    126
11     92
14     86
16     62
17     75
dtype: int64

In [180]:
with open('../ov2295_tree.pickle', 'rb') as f:
    clonal_tree = pickle.load(f)
clonal_tree.newick_str()

'((((3 10,4 11)2,(6 1,7 8)5)1,((10 0,11 17)9,((14 6,15 16)13,16 14)12)8)0);'

In [235]:
df_cell_meta['sample_id'].unique()

array(['SA1090', 'SA921', 'SA922'], dtype=object)